# BIO-SELECT - Marigliano
## Features merging using several lists

_TODO_ : insert global pipeline image here + highlight this notebook on the picture

## Imports

In [ ]:
from sklearn import neighbors, datasets
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from utils.ConfusionMatrix import ConfusionMatrix

import itertools
from sklearn.metrics import confusion_matrix

import math

%matplotlib inline

# set float precision at 2 digits
np.set_printoptions(precision=2)

# set the random seed for reproducibility
np.random.seed(0)

## Load the features lists

TODO: load the features lists from CSV files

In [ ]:
from utils.CSVFeaturesImporter import CSVFeaturesImporter

group_name = "golub"

importer = CSVFeaturesImporter(group_name)
subsets = importer.load()
print(subsets["features"].keys())
print(subsets["features_by_score"]["ReliefF"][:5])


features = subsets["features"]

## Features subsets merging
Each algorithm has done its work and provide a subset of features as:
* a ranked score list
* a ranked list (no score)
* a list (no ranking, no score)

This part uses some techniques to combine/merge theses lists into a better one

_TODO_: 
* Visualize the lists
    * Venn diagram ? --> limited to 3 sets, does not scale
    * matrix: show the similarity of features between two subsets
        * Jaccard
        * Union
* implement merge techniques
    * votation
    * weighted votation
    * union of intersection
    * ...

### Subsets visualization

In [ ]:
from scipy import spatial

# some set similarity functions
def intersection_count(a, b):
    return len(a.intersection(b))

def jaccard(a, b):
    return len(a.intersection(b))/float(len(a.union(b)))

def compute_similary_between_subsets(subsets, compare_func):
    N_subsets = len(subsets)
    similarity_matrix = np.zeros(shape=(N_subsets, N_subsets))

    for i, j in itertools.product(range(N_subsets), range(N_subsets)):
        if isinstance(subsets[0][0], int):
            subset_i = set(subsets[i])
            subset_j = set(subsets[j])
        else:
            subset_i = {i[0] for i in subsets[i]}
            subset_j = {j[0] for j in subsets[j]}

        similarity_matrix[i, j] = compare_func(subset_i, subset_j)
        
    return similarity_matrix

def plot_feature_subsets_matrix(cm, alg_names, title, cmap=plt.cm.Blues):
    title += "\n" # add a little margin for the title
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(alg_names))
    plt.xticks(tick_marks, alg_names, rotation=45)
    plt.yticks(tick_marks, alg_names)

    thresh = cm.max() / 2.0 + 0.1
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        text = "%.2f" % cm[i, j]
        plt.text(j, i, text,
                 horizontalalignment="center",
                 backgroundcolor="white",
                 #color="white" if cm[i, j] > thresh else "black")
                 color="black")

    plt.tight_layout()

# plot the similarity matrices
it = features.items()
alg_names = [f[0] for f in it]
subsets = [f[1] for f in it]

plt.figure(figsize=(12, 12))

similarity_matrix = compute_similary_between_subsets(subsets, compare_func=jaccard)
plt.subplot(1,2,1)
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Jaccard similarity between two feature subsets")

similarity_matrix = compute_similary_between_subsets(subsets, compare_func=intersection_count)
plt.subplot(1,2,2)
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Intersection between two feature subsets")

### Subsets merging

In [ ]:
from merge.simple.SimpleUnionSubsetMerger import SimpleUnionSubsetMerger

susm = SimpleUnionSubsetMerger(subsets)
merged_features = susm.merge()
n_all_features = sum([len(s) for s in subsets])

print("Unique features (union of all subsets): %d over a total of %d " % (len(merged_features), n_all_features))

## Evaluation of the merged subset
Once we have a merged list containing the best features, we would like to evaluate it with several classifiers

_TODO_: use a separate test set ? -> split again train/test set -> no changes in the Dataset class

### Dataset loading
_TODO_: 
* this notebook must only load one dataset
* retrieve dataset to load from cmd arguments or from env variable

In [ ]:
from datasets.EGEOD22619.EGEOD22619Dataset import EGEOD22619Dataset
from datasets.MILE.MileDataset import MileDataset
from datasets.Golub99.GolubDataset import GolubDataset

from datasets.DatasetEncoder import DatasetEncoder
from datasets.DatasetSplitter import DatasetSplitter
from datasets.DatasetLoader import DatasetLoader
from datasets.DatasetBalancer import DatasetBalancer

# Load dataset from environment variable. This is used by automated scripts
ds_class = DatasetLoader.load_from_env_var(default_dataset="Golub")

print("Dataset used: %s" % ds_class.__name__)

ds = ds_class()

### Dataset transformation
The dataset needs some transformations such as encoding the outputs as float (necessary for scikit learn), normalization, ...

_TODO_:
* dataset splitting (train, test[, validation])
* encode outputs
* normalization
* classes merging
    * due to the low class balancing we might want to regroup them. Example Healthy vs Non-Healthy (choose the most represented class ?)

In [ ]:
# encode Dataset string classes into numbers
ds_encoder = DatasetEncoder(ds)
ds = ds_encoder.encode()

ds_balancer = DatasetBalancer(ds)
ds = ds_balancer.balance()

ds = DatasetSplitter(ds, test_size=0.4)

X = ds.get_X()
y = ds.get_y()

X_train = ds.get_X_train()
y_train = ds.get_y_train()
X_test = ds.get_X_test()
y_test = ds.get_y_test()

class_names = range(len(set(ds.get_y())))

print("Number of genes: %d" % len(X_train[0]))
print("Dataset samples: %d" % len(y))
print("Train set size %d" % len(X_train))
print("Test set size %d" % len(X_test))

In [ ]:
is_list_unique = len(merged_features) == len(set(merged_features))
print("is list unique", is_list_unique)

merged_features = list(merged_features)

### Assess merged features

TODO: compare performance against a list of random features

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier


def assess_merged_features(clf, clf_name, assessment_scores):
    clf.fit(ds.get_X_train(), ds.get_y_train())
    y_pred = clf.predict(ds.get_X_test())
    y_test = ds.get_y_test()

    scores = cross_val_score(clf, ds.get_X_test()[:, merged_features], ds.get_y_test(), cv=3, n_jobs=-1)
    score = np.mean(scores)

    print("[%s] Score using the merged list of features: %.3f" % (clf_name, score))

    assessment_scores[clf_name] = score, (y_test, y_pred)

    
assessment_scores  = {}

clf = KNeighborsClassifier(n_neighbors=5)
assess_merged_features(clf, "KNN", assessment_scores)

clf = MLPClassifier(solver="adam", alpha=1e-3, hidden_layer_sizes=(100, 50), activation="relu")
#clf = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', early_stopping=False,
#       epsilon=1e-08, hidden_layer_sizes=(100,50), learning_rate='constant',
#       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
#       nesterovs_momentum=True, power_t=0.5, shuffle=True,
#       solver='lbfgs', tol=0.0001, verbose=False,
#       warm_start=False)

assess_merged_features(clf, "MLP", assessment_scores)

clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100)
assess_merged_features(clf, "ExtraTrees", assessment_scores)

### Confusion Matrix

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import math

plt.figure(figsize=(12, 8))

n_subplots = len(assessment_scores)
cols = 3
rows = int(math.ceil(n_subplots / cols))
i = 1

for name, score_cm in assessment_scores.iteritems():
    y_test, y_pred = score_cm[1]
    cnf_matrix = confusion_matrix(y_test, y_pred)

    plt.subplot(rows, cols, i)
    i += 1

    ConfusionMatrix.plot(cnf_matrix, classes=class_names,
                          title='Confusion matrix for %s' % name)

plt.tight_layout()
plt.show()